In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold

MAPPING THE LABELS

In [10]:
pass

LOAD THE SAVED MODEL

In [2]:
feature_extractor = torch.hub.load(
        'OxWearables/ssl-wearables', 
        'harnet5', 
        pretrained=True).feature_extractor

print(feature_extractor)

Using cache found in C:\Users\jgm_6/.cache\torch\hub\OxWearables_ssl-wearables_main


131 Weights loaded
Sequential(
  (layer1): Sequential(
    (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
    (1): ResBlock(
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (relu): ReLU(inplace=True)
    )
    (2): ResBlock(
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), pa

C:\Users\jgm_6/.cache\torch\hub\OxWearables_ssl-wearables_main\hubconf.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(weight_path, map_loc

In [3]:
for param in feature_extractor.parameters():
    param.requires_grad = False  # Freeze feature extractor

In [4]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

- Best trial:
- Value: 0.7475
- Params:
  - lstm_layers: 2
  - hidden_size: 256
  - dropout: 0.26138219898271
  - activation: ReLU
  - lr: 0.0002829985470448175
  - weight_decay: 2.1242103361847734e-05
  - batch_size: 64
  - optimizer: Adam


In [5]:
# Define the model
class CustomModel(nn.Module):
    def __init__(self, feature_extractor, lstm_layers=1, hidden_size=256, dropout=0.2, activation='ReLU'):
        super(CustomModel, self).__init__()
        self.feature_extractor = feature_extractor

        # Add a Linear layer to transform (batch_size, seq_length, 1) -> (batch_size, seq_length, 512)
        self.feature_transformer = nn.Linear(1, 512)

        self.lstm = nn.LSTM(input_size=512, hidden_size=hidden_size, num_layers=lstm_layers,
                            batch_first=True, dropout=dropout if lstm_layers > 1 else 0)
        
        activation_layer = {
            'ReLU': nn.ReLU(),
            'Tanh': nn.Tanh(),
            'LeakyReLU': nn.LeakyReLU()
        }[activation]
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            activation_layer,
            nn.Linear(512, 1024),
            activation_layer,
            nn.Linear(1024, 512),
            activation_layer,
            nn.Linear(512, 10),
            # activation_layer,
            # nn.Linear(256, 206)
        )
    
    def forward(self, x):
        features = self.feature_extractor(x)
        # print("Feature extractor output shape:", features.shape)  # Debugging line to check the shape
        features = self.feature_transformer(features)
        
        lstm_out, _ = self.lstm(features)
        lstm_last_output = lstm_out[:, -1, :]
        output = self.classifier(lstm_last_output)
        return output

In [7]:
# Create a new instance of the classifier
classifier = CustomModel(
    feature_extractor=feature_extractor,
    lstm_layers=2,
    hidden_size=256,
    dropout=0.26138219898271,
    activation='ReLU'
)

# Load the state dictionary into the new instance
calssifier_path = r'D:\Rainbow_DQN_Test\Test_DQN\Data\process_data\best_model_10_meta_labels.pth'
classifier.load_state_dict(torch.load(calssifier_path))
# classifier.eval()  # Set the model to evaluation mode

# Now `classifier` contains the saved weights
print("Model loaded successfully!")

Model loaded successfully!


C:\Users\jgm_6\AppData\Local\Temp\ipykernel_18872\734222708.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(calssifier_path))


LOAD 5 SECONDS SEQUENCES FROM H5PY DATASET

In [8]:
def read_h5(filepath, dataset_name):
    with h5py.File(filepath, 'r') as h5file:
        data = h5file[dataset_name][:]
    return data

In [9]:
X30 = read_h5(r'D:\Rainbow_DQN_Test\Test_DQN\Data\Process_canada_data\P13_5_sec_30hz_sequences.h5', 'data')
X30.shape

(417840, 8, 150)

In [11]:
X_acc = X30[:, [2, 3, 4], :]  # Shape becomes (417840, 3, 320)
X_acc

array([[[-0.50025874, -0.50025874, -0.50025874, ..., -0.50267789,
         -0.49709149, -0.50343321],
        [ 0.15633085,  0.15633085,  0.15633085, ...,  0.16501594,
          0.1678173 ,  0.14978904],
        [ 0.82855353,  0.82855353,  0.82855353, ...,  0.8339162 ,
          0.84018767,  0.82172327]],

       [[-0.50025874, -0.48462565, -0.50025874, ..., -0.49865015,
         -0.50316705, -0.48893228],
        [ 0.15633085,  0.17196394,  0.15633085, ...,  0.17262687,
          0.17304723,  0.16428411],
        [ 0.82855353,  0.84418662,  0.84418662, ...,  0.83896935,
          0.83700964,  0.8244901 ]],

       [[-0.48462565, -0.50025874, -0.48462565, ..., -0.48773283,
         -0.48779889, -0.5060838 ],
        [ 0.15633085,  0.15633085,  0.15633085, ...,  0.21929406,
          0.21948304,  0.21223335],
        [ 0.82855353,  0.82855353,  0.82855353, ...,  0.82739054,
          0.82804227,  0.82722642]],

       ...,

       [[-0.73475502, -0.73475502, -0.73475502, ..., -0.7156842

ONE SEQUENCE DID WORK !!

In [13]:
import numpy as np
import torch

all_logits = []

# Example: A single sequence from your dataset (replace with your actual data)
# Shape: (8, 320) for one sequence
single_sequence = X_acc[0].astype(np.float32)

# Convert to PyTorch tensor and add a batch dimension
single_sequence_tensor = torch.tensor(single_sequence, dtype=torch.float32).unsqueeze(0)  # Shape: (1, 3, 320)

# Move model and data to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = classifier.to(device)
single_sequence_tensor = single_sequence_tensor.to(device)

# Perform inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(single_sequence_tensor)  # Shape: (1, num_classes)


all_logits.extend(output.cpu().tolist())  # Convert to list and store

# Get predicted class
predicted_class = torch.argmax(output, dim=-1).item()
print(f"Predicted class for the sequence: {predicted_class}")


Predicted class for the sequence: 8


In [14]:
all_logits

[[-1.7843915224075317,
  -8.75704574584961,
  -9.047948837280273,
  0.5426174998283386,
  -7.941391468048096,
  -4.290830612182617,
  -5.96685266494751,
  -8.919855117797852,
  1.9559062719345093,
  -8.95846939086914]]

In [15]:
# Convert to tensor
all_logits_tensor = torch.tensor(all_logits)

# Find the highest class and logit for each sequence
sequence_highest_classes = torch.argmax(all_logits_tensor, dim=-1).tolist()
sequence_highest_logits = torch.max(all_logits_tensor, dim=-1).values.tolist()

In [16]:
# Output
print(sequence_highest_classes)  # [3, 1, 1]
print(sequence_highest_logits)   # [4.2, 3.4, 2.2]

[8]
[1.9559062719345093]


MORE SEQUENCES ...

In [18]:
sequences = X_acc.astype(np.float32)

In [19]:
sequences.shape[0]

417840

In [20]:
# Define the required parameters
all_logits = []
results = []

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = classifier.to(device)

# Set the model to evaluation mode
model.eval()

# Process data in batches
with torch.no_grad():
    for i in range(sequences.shape[0]):
        sequence = sequences[i]
        
        # Convert batch to PyTorch tensor
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)
        
        # Move data and model to the same device
        sequence_tensor = sequence_tensor.to(device)
        
        # Classify the batch
        output = model(sequence_tensor)

        # Store logits for this batch
        all_logits.extend(output.cpu().tolist())  # Convert to list and store

        predicted_class = torch.argmax(output, dim=-1).item()

        # Append results
        results.append(predicted_class)

print("Classification completed using batches.")


Classification completed using batches.


In [21]:
logits_array = np.array(all_logits)
results_array = np.array(results)

In [23]:
logits_array[0]

array([-1.78439152, -8.75704575, -9.04794884,  0.5426175 , -7.94139147,
       -4.29083061, -5.96685266, -8.91985512,  1.95590627, -8.95846939])

In [25]:
results_array[0]

8

SAVE IN NPZ

In [26]:
np.savez_compressed('P13_classifications', array=results_array)

In [27]:
np.savez_compressed('P13_class_probs', array=logits_array)